<a href="https://colab.research.google.com/github/highway92/machine_learning/blob/main/year_dream/do/vggnet16_augmentation_withL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
# !apt-get update && apt-get install -y python3-opencv

In [ ]:
# !pip install sklearn

In [ ]:
# !pip install opencv-python

In [ ]:
# !pip install pandas

In [ ]:
# !pip install wandb

In [ ]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

In [ ]:
# wandb.init(project = 'test-project', entity = 'pnm-team')
# wandb.log({"loss": 1})
# wandb.finish()

## Set Arguments & hyperparameters

In [ ]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [ ]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

## Dataloader

#### Train & Validation Set loader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape,aug_mode):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        self.aug_mode = aug_mode
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        if self.aug_mode == "normal":
            self.transform = transforms.Compose([
                                                 transforms.Resize(self.input_shape),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        elif self.aug_mode == "radom_crop":
            self.transform = transforms.Compose([
                                                 transforms.RandomCrop(self.input_shape),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        elif self.aug_mode == 'H_flip':
            self.transform = transforms.Compose([
                                                 transforms.Resize(self.input_shape),
                                                 transforms.RandomHorizontalFlip(),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        elif self.aug_mode == 'V_flip':
            self.transform = transforms.Compose([
                                                 transforms.Resize(self.input_shape),
                                                 transforms.RandomVerticalFlip(),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        elif self.aug_mode == 'rotate':
            self.transform = transforms.Compose([
                                                 transforms.Resize(self.input_shape),
                                                 transforms.RandomRotation([-180, 180]),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']

## Model

In [ ]:
import torch.nn.functional as F

VGG_types = {
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
}

class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=2, init_weights=True):
        super(VGGnet, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_laters(VGG_types[model])
        
        self.fcs = nn.Sequential(
            nn.Linear(512*4*4, 2048),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes)
        )
        self.softmax = nn.Softmax(dim=1)
        
        if init_weights:
            self._initialize_weights()
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 512*4*4)
        x = self.fcs(x)
        output = self.softmax(x)
        return output
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out',
                                       nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def create_conv_laters(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == int:
                out_channels = x
                
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                    kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                          nn.BatchNorm2d(x),
                          nn.ReLU()]
                in_channels = x
            
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        
        return nn.Sequential(*layers)

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
        
#         return output

In [ ]:
# # x = torch.randn(3,3,225,225).to(DEVICE)
# model = VGGnet('VGG16').to(DEVICE)
# model(x)

## Utils
### EarlyStopper by loss

In [ ]:
# class LossEarlyStopper():
#     """Early stopper
    
#     Attributes:
#         patience (int): loss가 줄어들지 않아도 학습할 epoch 수
#         patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
#         min_loss (float): 최소 loss
#         stop (bool): True 일 때 학습 중단

#     """

#     def __init__(self, patience: int)-> None:
#         self.patience = patience

#         self.patience_counter = 0
#         self.min_loss = np.Inf
#         self.stop = False
#         self.save_model = False

#     def check_early_stopping(self, loss: float)-> None:
#         """Early stopping 여부 판단"""  

#         if self.min_loss == np.Inf:
#             self.min_loss = loss
#             return None

#         elif loss > self.min_loss:
#             self.patience_counter += 1
#             msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

#             if self.patience_counter == self.patience:
#                 self.stop = True
                
#         elif loss <= self.min_loss:
#             self.patience_counter = 0
#             self.save_model = True
#             msg = f"Validation loss decreased {self.min_loss} -> {loss}"
#             self.min_loss = loss
        
#         print(msg)

# Early Stoppper by acc

In [ ]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.max_acc = -np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, acc: float)-> None:
        """Early stopping 여부 판단"""  

        if self.max_acc == -np.Inf:
            self.max_acc = acc
            return None

        elif acc < self.max_acc:
            self.patience_counter += 1
            self.save_model = False
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif acc >= self.max_acc:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation acc increased {self.max_acc} -> {acc}"
            self.max_acc = acc
        
        print(msg)

### Trainer

In [ ]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [ ]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE, aug_mode = 'normal')
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE, aug_mode = 'normal')
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


In [ ]:
aug_modes = ['normal','radom_crop', 'H_flip', 'V_flip', 'rotate']
all_datasets = []
all_validsets = []
for mode in aug_modes:
    d_set = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE, aug_mode = mode)
    all_datasets.append(d_set)

for mode in aug_modes:
    d_set = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE, aug_mode = mode)
    all_validsets.append(d_set)
    
train_dataset = torch.utils.data.ConcatDataset(all_datasets)
validation_dataset = torch.utils.data.ConcatDataset(all_validsets)


Loading train dataset..
Loading train dataset..
Loading train dataset..
Loading train dataset..
Loading train dataset..
Loading val dataset..
Loading val dataset..
Loading val dataset..
Loading val dataset..
Loading val dataset..


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Train set samples: 2905 Val set samples: 325


In [ ]:
img, _ = train_dataset[1]
print(img.shape)

torch.Size([3, 128, 128])


#### Load model and other utils

In [ ]:
# Load Model
model = VGGnet('VGG16', num_classes=NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [ ]:
model

VGGnet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), 

### epoch 단위 학습 진행

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# DATA_DIR= 'data'
# NUM_CLS = 2

# EPOCHS = 30
# BATCH_SIZE = 32
# LEARNING_RATE = 0.0005
# EARLY_STOPPING_PATIENCE = 10
# INPUT_SHAPE = 128
import wandb

wandb.init(project = 'week1_Image', entity = 'pnm-team',name = "VGG16_vaild_aug_L2",group = 'VGG', config={
    'lr' : LEARNING_RATE,
    'epochs' : EPOCHS,
    'batch_size' : BATCH_SIZE,
    'input_shape' : INPUT_SHAPE
})


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pnm (use `wandb login --relogin` to force relogin)


In [ ]:
for epoch_index in tqdm(range(30)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)
    wandb.log({'train_loss' : trainer.train_mean_loss,
               'validation_loss' : trainer.val_mean_loss,
               'train_acc' : trainer.train_score,
               'validation_acc' : trainer.validation_score})
    # early_stopping check
    early_stopper.check_early_stopping(acc = trainer.validation_score)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')

  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.70093744330936, Acc: 0.47986230636833044, F1-Macro: 0.45748415663227626


  3% 1/30 [04:44<2:17:18, 284.08s/it]

Epoch 0, Val loss: 0.7624521911144256, Acc: 0.5169230769230769, F1-Macro: 0.3905352301095305
Epoch 1, Train loss: 0.7004551397429573, Acc: 0.5487091222030981, F1-Macro: 0.528035119509463
Epoch 1, Val loss: 0.7615539014339447, Acc: 0.5476923076923077, F1-Macro: 0.4706312535318951
Validation acc increased 0.5169230769230769 -> 0.5476923076923077


  7% 2/30 [10:12<2:18:43, 297.25s/it]

...

Epoch 20, Val loss: 0.7695200502872467, Acc: 0.72, F1-Macro: 0.7199045355103277
Early stopping counter 8/10
Epoch 21, Train loss: 0.21451051897472806, Acc: 0.9005163511187607, F1-Macro: 0.8998211434907937


 73% 22/30 [2:02:50<35:52, 269.07s/it]

Epoch 21, Val loss: 0.7000075556337834, Acc: 0.7138461538461538, F1-Macro: 0.7127583749109052
Early stopping counter 9/10
Epoch 22, Train loss: 0.2043658343454202, Acc: 0.9070567986230637, F1-Macro: 0.9067182817182817


 73% 22/30 [2:06:55<46:09, 346.17s/it]

Epoch 22, Val loss: 0.8204008877277374, Acc: 0.6953846153846154, F1-Macro: 0.6927667701121986
Early stopping counter 10/10
Early stopped


## Inference
### 모델 로드

In [ ]:
wandb.finish()

train_acc,▁▂▃▅▅▆▆▆▆▇▇▇▇█▇████████
train_loss,███▆▆▅▅▄▄▃▃▂▃▂▂▂▂▂▂▁▁▁▁
validation_acc,▁▂▅▇▆▇▄▅▆▆█▆█▆▇▆▆█▇█▇▇▇
validation_loss,▃▃▂▂█▁▂▃▅▅▂▃▃▄▃▆█▂▃▄▃▂▄
train_acc,0.90706
train_loss,0.20437
validation_acc,0.69538
validation_loss,0.8204


In [ ]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [ ]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [ ]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:01,  1.57s/it]

tensor([[9.9911e-01, 8.8869e-04],
        [7.7043e-01, 2.2957e-01],
        [2.3372e-01, 7.6628e-01],
        [1.8680e-01, 8.1320e-01],
        [9.9992e-01, 8.4431e-05],
        [1.2321e-01, 8.7679e-01],
        [2.0412e-01, 7.9588e-01],
        [8.6829e-01, 1.3171e-01],
        [9.9456e-01, 5.4374e-03],
        [3.9663e-03, 9.9603e-01],
        [9.9941e-01, 5.9012e-04],
        [9.8077e-01, 1.9234e-02],
        [9.9956e-01, 4.4463e-04],
        [2.0804e-01, 7.9196e-01],
        [9.8812e-01, 1.1876e-02],
        [9.9869e-01, 1.3110e-03],
        [9.9184e-01, 8.1572e-03],
        [9.9851e-01, 1.4910e-03],
        [9.9973e-01, 2.6939e-04],
        [9.9813e-01, 1.8698e-03],
        [9.9999e-01, 6.3140e-06],
        [9.9655e-01, 3.4538e-03],
        [9.3449e-01, 6.5507e-02],
        [9.9996e-01, 4.3025e-05],
        [1.3920e-01, 8.6080e-01],
        [9.9990e-01, 1.0319e-04],
        [9.9996e-01, 4.3913e-05],
        [3.5414e-01, 6.4586e-01],
        [4.1676e-03, 9.9583e-01],
        [1.757

2it [00:03,  1.76s/it]

tensor([[1.9506e-02, 9.8049e-01],
        [9.9989e-01, 1.1437e-04],
        [9.9908e-01, 9.2365e-04],
        [8.4223e-01, 1.5777e-01],
        [9.6062e-01, 3.9378e-02],
        [5.4982e-01, 4.5018e-01],
        [1.1816e-02, 9.8818e-01],
        [9.9157e-01, 8.4310e-03],
        [6.1721e-01, 3.8279e-01],
        [9.8184e-01, 1.8163e-02],
        [9.8325e-01, 1.6747e-02],
        [1.6275e-03, 9.9837e-01],
        [6.6470e-01, 3.3530e-01],
        [2.3743e-01, 7.6257e-01],
        [2.4738e-01, 7.5262e-01],
        [4.2999e-01, 5.7001e-01],
        [1.1633e-01, 8.8367e-01],
        [9.9149e-01, 8.5109e-03],
        [5.8893e-01, 4.1107e-01],
        [9.6802e-03, 9.9032e-01],
        [9.2450e-02, 9.0755e-01],
        [9.9375e-01, 6.2507e-03],
        [9.0899e-01, 9.1008e-02],
        [1.2821e-01, 8.7179e-01],
        [8.9474e-01, 1.0526e-01],
        [8.8319e-01, 1.1681e-01],
        [2.8759e-01, 7.1241e-01],
        [7.3660e-01, 2.6340e-01],
        [7.1898e-03, 9.9281e-01],
        [9.999

4it [00:05,  1.49s/it]

tensor([[2.6554e-02, 9.7345e-01],
        [7.7259e-04, 9.9923e-01],
        [1.7740e-01, 8.2260e-01],
        [4.9192e-02, 9.5081e-01],
        [9.8544e-01, 1.4557e-02],
        [9.9607e-01, 3.9339e-03],
        [9.8066e-01, 1.9336e-02],
        [7.2136e-01, 2.7864e-01],
        [6.5082e-01, 3.4918e-01],
        [9.9989e-01, 1.0801e-04],
        [2.3926e-03, 9.9761e-01],
        [3.7997e-01, 6.2003e-01],
        [9.9971e-01, 2.8567e-04],
        [9.9910e-01, 8.9819e-04],
        [1.8224e-01, 8.1776e-01],
        [9.5346e-03, 9.9047e-01],
        [1.0586e-02, 9.8941e-01],
        [9.8910e-01, 1.0904e-02],
        [9.9669e-01, 3.3130e-03],
        [2.6780e-02, 9.7322e-01],
        [9.3147e-02, 9.0685e-01],
        [1.4807e-01, 8.5193e-01],
        [9.8782e-01, 1.2185e-02],
        [9.3377e-01, 6.6226e-02],
        [9.8774e-01, 1.2258e-02],
        [8.9350e-01, 1.0650e-01],
        [9.7992e-01, 2.0084e-02],
        [9.8216e-01, 1.7844e-02],
        [3.8387e-03, 9.9616e-01],
        [1.753

### 결과 저장

In [ ]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('vgg16_2.csv', index=False)